In [39]:
from datasets import load_dataset
import datasets
from collections import Counter
import re
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import tokenizers
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [26]:
# 加载 IMDB 数据集
cache_dir=r'.\data_file'
imdb_dataset = load_dataset("imdb",cache_dir=cache_dir)
# .map(): 对所有样本应用一个函数 (非常重要，用于预处理):

In [37]:
imdb_dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [40]:
class IMDBDataset(Dataset):
    def __init__(self, data_type):
        super(IMDBDataset, self).__init__()
        self.dataset=imdb_dataset[data_type]
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

In [41]:
imdb_data=IMDBDataset('train')

In [2]:
from transformers import AutoModel, AutoTokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [74]:
pretrain_model = AutoModel.from_pretrained('bert-base-uncased',cache_dir=r'D:\bigdata_project\models_file')
pretrain_model=pretrain_model.to(device)
token_tool = AutoTokenizer.from_pretrained('bert-base-uncased',cache_dir=r'D:\bigdata_project\models_file')

In [76]:
token_tool.tokenize(x)

['i',
 'rented',
 'i',
 'am',
 'curious',
 '-',
 'yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.',
 'i',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'u',
 '.',
 's',
 '.',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country',
 ',',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 '"',
 'controversial',
 '"',
 'i',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself',
 '.',
 'the',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life',
 '.',
 'in',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attention',
 '##s',
 'to',
 'making',
 'some',
 'sort',
 'of',
 'documentary',
 'on',
 'what',
 'the',
 'ave

In [65]:
import re
from bs4 import BeautifulSoup  # 用于去除HTML标签

def clean_text(text):
    # 去除HTML标签（常见于IMDB数据集）
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 替换或删除特殊字符（保留基本标点）
    text = re.sub(r"@[\w]+", "", text)                # 移除@提及
    text = re.sub(r"http\S+", "", text)               # 移除URL
    # text = re.sub(r"[^a-zA-Z0-9!?.,:;'\"\-]", " ", text)  # 保留基本字符，其他替换为空格
    text = re.sub(r"\s+", " ", text)                  # 合并多个空格
    
    # 可选：处理缩写（如 don't → do not）
    text = text.replace("n't", " not").replace("'s", " is").strip().lower()
    
    return text.strip()

def token_embed(text):
    text=clean_text(text)
    return token_tool.convert_tokens_to_ids(token_tool(text))
    

In [68]:
token_embed(imdb_dataset['train'][0]['text'])

[100, 100, 100]

In [79]:
x=clean_text(imdb_dataset['train'][0]['text'])
token_tool(x,max_length=512)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 3836, 1010, 19846, 1010, 1998, 2496, 2273, 1012, 20

In [30]:
DataLoader(imdb_data,batch_size=32,shuffle=True,drop_last=True,collate_fn=token_tool)

['i',
 'rented',
 'i',
 'am',
 'curious',
 '-',
 'yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.',
 'i',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'u',
 '.',
 's',
 '.',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country',
 ',',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 '"',
 'controversial',
 '"',
 'i',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself',
 '.',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'the',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life',
 '.',
 'in',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attention',
 '##s',
 'to',
 'making',
 'some',
 'sort',
 '

In [24]:
imdb_dataset

NameError: name 'imdb_dataset' is not defined